In [0]:
# Create the raw Delta table with CDC enabled
spark.sql("""
CREATE TABLE IF NOT EXISTS incremental_load.default.raw_upi_transactions(
    transaction_id STRING,
    upi_id STRING,
    merchant_id STRING,
    transaction_amount DOUBLE,
    transaction_timestamp TIMESTAMP,
    transaction_status STRING
)
USING DELTA
TBLPROPERTIES ('delta.enableChangeDataFeed' = 'true')
""")
print("Delta table 'incremental_load.default.raw_upi_transactions' created with CDC enabled.")

In [0]:
from delta.tables import DeltaTable
import time

mock_batches=[
    spark.createDataFrame([
        ("T001", "upi1@bank", "M001", 500.0, "2024-12-21 10:00:00", "initiated"),
        ("T002", "upi2@bank", "M002", 1000.0, "2024-12-21 10:05:00", "initiated"),
        ("T003", "upi3@bank", "M003", 1500.0, "2024-12-21 10:10:00", "initiated"),
    ], ["transaction_id", "upi_id", "merchant_id", "transaction_amount", "transaction_timestamp", "transaction_status"]),
    spark.createDataFrame([
        ("T001", "upi1@bank", "M001", 500.0, "2024-12-21 10:15:00", "completed"),  # Update transaction
        ("T002", "upi2@bank", "M002", 1000.0, "2024-12-21 10:20:00", "failed"),    # Update transaction
        ("T004", "upi4@bank", "M004", 2000.0, "2024-12-21 10:25:00", "initiated"),
    ], ["transaction_id", "upi_id", "merchant_id", "transaction_amount", "transaction_timestamp", "transaction_status"]),
    spark.createDataFrame([
        ("T001", "upi1@bank", "M001", 500.0, "2024-12-21 10:30:00", "refunded"),  # Refund issued
        ("T003", "upi3@bank", "M003", 1500.0, "2024-12-21 10:35:00", "completed"),
    ], ["transaction_id", "upi_id", "merchant_id", "transaction_amount", "transaction_timestamp", "transaction_status"])
]

def merge_to_delta_table(delta_table_name:str, batch_df):
    delta_table = DeltaTable.forName(spark, delta_table_name)
    delta_table.alias("target").merge(
        batch_df.alias("source"),
        "target.transaction_id = source.transaction_id"
    ).whenMatchedUpdate(
        set={
            "upi_id": "source.upi_id",
            "merchant_id": "source.merchant_id",
            "transaction_amount": "source.transaction_amount",
            "transaction_timestamp": "source.transaction_timestamp",
            "transaction_status": "source.transaction_status"
        }
    ).whenNotMatchedInsertAll().execute()

merge_to_delta_table("incremental_load.default.raw_upi_transactions", mock_batches[0])
print(f"Batch processed successfully.")
